0. Import Library

In [ ]:
import math
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler, ConcatDataset, Dataset
from torch.utils.data.dataset import random_split
from torcheval.metrics.functional import binary_auroc
from sklearn.metrics import roc_auc_score

from utils.rnn_attention_kd import *

1. Checking the Device

In [ ]:
# Checking whether the device used for training is CPU or GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

2. Import the Teacher Model

In [ ]:
# Setting the hyperparametes of teacher model.
input_size_teacher = 12 # Number of feature on this study
hidden_size_teacher = 4 # The hidden units on each layer on RNN
num_layers_teacher = 1 # The number of layers of RNN
num_classes_teacher = 2 # The number of class (At-risk and No-risk students)

In [ ]:
# Setting a model
model_t = TeacherGRU(input_size=input_size_teacher, 
                     hidden_size=hidden_size_teacher, 
                     num_layers=num_layers_teacher, 
                     num_classes=num_classes_teacher)

model_t = model_t.to(device)

In [ ]:
# Loading a save weight of teacher model 
PATH_T = f'save/{file_path_teacher}/teacher_model'
model_t.load_state_dict(torch.load(PATH_T))

3. Training a Student Model

In [ ]:
# Setting the hyperparametes of student model.
input_size_student = 12 # Number of feature on this study
hidden_size_student = 4 # The hidden units on each layer on RNN
num_layers_student = 1 # The number of layers of RNN
num_classes_student = 2 # The number of class (At-risk and No-risk students)

In [ ]:
model_s = StudentGRU(input_size=input_size_student, 
                     hidden_teacher_size=hidden_size_teacher, 
                     hidden_student_size=hidden_size_student, 
                     num_layers=num_layers_student, 
                     num_classes=num_classes_student)

model_s = model_s.to(device)

In [ ]:
# Training the student model
train_student(model_t=model_t, 
              model_s=model_s, 
              train_loader=train_kd_loader,
              epochs=150, 
              device=device, 
              train_graph_path=None, 
              learning_rate=0.01, 
              weight_decay=1e-5,
              scheduler_funnction=False, 
              print_results=False,
              alpha=0.1)

In [ ]:
# Evaluation the performance of student model with test set
precision, recall, f1, accuracy, auc, predict_value = model_evaluation_student(model=model_s, 
                                                                               test_loader=test_kd_loader, 
                                                                               device=device, 
                                                                               threshold=0.5,
                                                                               confusion_matrix=False, 
                                                                               print_results=False)

In [ ]:
# Save the student weight model
PATH_S = f'save/{file_path_student}/student_model'
torch.save(model_s.state_dict(), PATH_S)